<a href="https://colab.research.google.com/github/kr7/IntelligensModszerekTantargy/blob/main/Konvolucios_halo_tanitasa_GPU_n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim

from google.colab import widgets

**Adatok betöltése és megjelenítése**

In [2]:
data = np.loadtxt('https://archive.ics.uci.edu/ml/machine-learning-databases/semeion/semeion.data')

Felosztjuk az adatainkat tanító és teszt adatokra. Az utolsó 500 példány lesz a teszt adathalmaz, a többi a tanítóhalmaz.

In [3]:
train_data = data[:1093,0:256]
train_labels = data[:1093,256:266]
test_data = data[1093:,0:256]
test_labels = data[1093:,256:266]

In [4]:
class DigitRecognizerCNN(nn.Module):
    def __init__(self):
        super(DigitRecognizerCNN, self).__init__()
        num_units_fc = 1000

        self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 32, 
                               kernel_size=(5,5), padding='same')
        self.conv2 = nn.Conv2d(in_channels = 32, out_channels = 64, 
                               kernel_size=(5,5), padding='same')
        self.fc1 = nn.Linear(64*16*16, num_units_fc)
        self.fc2 = nn.Linear(num_units_fc, num_units_fc)
        self.out = nn.Linear(num_units_fc, 10) 

    def forward(self, x):
        x = x.view(-1, 1, 16, 16)
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(-1, 64*16*16)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.out(x)
        return x

Tanítás GPU-n, ha elérhető a GPU

In [5]:
gpu = torch.cuda.is_available()

In [6]:
print(gpu)

True


In [7]:
train_dataset = torch.utils.data.TensorDataset(
    torch.Tensor(train_data), torch.Tensor(train_labels) )
trainloader = torch.utils.data.DataLoader(
    train_dataset, shuffle=True, batch_size=16)

net = DigitRecognizerCNN()
if gpu:
  net.cuda()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01)

running_loss = 0.0
running_n = 0

best_acc = 0
best_epoch = 0

for epoch in range(100):  
  for inputs, targets in trainloader:
    optimizer.zero_grad()
    
    if gpu:
      inputs = inputs.cuda()
      targets = targets.cuda()

    outputs = net(inputs)

    loss = criterion(outputs, targets)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    running_n = running_n + 1

In [8]:
def eval_net(net):
  test_dataset = torch.utils.data.TensorDataset( 
        torch.Tensor(test_data), torch.Tensor(test_labels))
  testloader = torch.utils.data.DataLoader(test_dataset)

  correct = 0
  total = 0

  with torch.no_grad():
    for inputs, targets in testloader:
      if gpu:
        inputs = inputs.cuda()
        targets = targets.cuda()
        
      outputs = net(inputs)
      _, predicted = torch.max(outputs.data, 1)
      _, targets_ordinal = torch.max(targets, 1)
      
      # a "corrects" és a total változók a CPU-n vannak, ezért 
      # "vissza kell hozni" targets-et és a helyes klasszifikációk számát
      #  a CPU-ra
      targets = targets.cpu()
      correct += (predicted == targets_ordinal).cpu()
      total += targets.size(0)

  return float(correct/total)

In [9]:
eval_net(net)

0.9139999747276306